In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from my_utils import seq_10, DataLoader
from torch_models.utils import seq2label

device = 'cpu'
trans_func = seq2label(device)
train = seq_10(5000)
test = seq_10(500)
train_loader = DataLoader(train, batch_size=16, trans_func=trans_func)
test_loader = DataLoader(test, batch_size=64, trans_func=trans_func)

In [3]:
from torch_models.models import BoV, RNNLastHidden, RNNMaxPool, SingleClassifier
from torch_models.utils import get_device

encoder = RNNLastHidden(embed_size=10, hidden_size=15, vocab_size=10, bidirectional=True, num_layers=1, rnn='gru')
model = SingleClassifier(encoder=encoder, output_size=2, hidden_size=None,
                         activation='Tanh', dropout=0, freeze_encoder=False)
print(model)

SingleClassifier(
  (encoder): RNNLastHidden(
    (embedding): Embedding(11, 10, padding_idx=10)
    (rnn): GRU(10, 15, bidirectional=True)
  )
  (mlp): MLP(
    (fc_out): Linear(in_features=30, out_features=2, bias=True)
    (dropout): Dropout(p=0)
    (criterion): CrossEntropyLoss()
    (activation): Tanh()
  )
)


In [4]:
from my_utils import Trainer, EvaluatorC
from torch.optim import Adam

optimizer = Adam(model.parameters())

evaluator = EvaluatorC(model, test_loader)

trainer = Trainer(model, train_loader)
trainer.train(optimizer, max_epoch=1,
              evaluator=evaluator, score_monitor=None, show_log=True, hook_func=None)

epoch 0  	loss: 0.44477771743894007	accuracy: 0.928	
